In [60]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, RegularPolygon
from IPython.display import clear_output
import time
from abc import ABC, abstractmethod

# Environment settings
FIELD_SIZE = 150
MAX_FOOD_ITEMS = 200  # Maximum number of food items
FOOD_RESPAWN_INTERVAL = 50  # Number of steps after which food respawns
FOOD_ENERGY_VALUE = 10
SMALL_VARIATION = 0.2
LARGE_VARIATION = 0.5

# Utility function to calculate distance between two positions
def distance(pos1, pos2):
    return np.linalg.norm(pos1 - pos2)

class Field:
    """Class representing the simulation field."""
    def __init__(self, size):
        self.size = size

class Food:
    def __init__(self, position):
        self.position = np.array(position)

class Agent(ABC):
    """Abstract base class for all agents in the simulation."""
    def __init__(self, position):
        self.position = np.array(position, dtype='float')

    @abstractmethod
    def update(self):
        pass

class Animal(Agent):
    """Base class for animals with basic properties and methods."""
    def __init__(self, position, **kwargs):
        super().__init__(position)
        # Default properties
        self.size = kwargs.get('size', 2)
        self.speed = kwargs.get('speed', 1)
        self.energy = kwargs.get('energy', 100)
        self.max_energy = kwargs.get('max_energy', 150)
        self.radius_of_view = kwargs.get('radius_of_view', 20)
        self.direction_change_interval = kwargs.get('direction_change_interval', 10)
        self.color = kwargs.get('color', 'blue')
        self.generation = kwargs.get('generation', 1)
        self.eaten_count = 0
        self.steps_in_current_direction = 0
        self.current_direction = None

    def update(self, environment):
        """Update the animal's state."""
        if self.is_alive():
            self.move(environment)
            self.handle_boundary()
            self.energy -= 0.5  # Energy decreases over time

    def move(self, environment):
        """Determine movement behavior."""
        # Default movement is random
        self.random_move()

    def random_move(self):
        """Move in a random direction for a set number of steps."""
        if self.steps_in_current_direction == 0 or self.current_direction is None:
            angle = np.random.uniform(0, 2 * np.pi)
            self.current_direction = np.array([np.cos(angle), np.sin(angle)])
            self.steps_in_current_direction = self.direction_change_interval

        self.position += self.current_direction * self.speed
        self.steps_in_current_direction -= 1

    def is_alive(self):
        """Check if the animal is still alive (has energy greater than 0)."""
        return self.energy > 0

    def handle_boundary(self):
        """Handle reflection when the animal reaches the boundary."""
        for i in [0, 1]:  # 0 for x-axis, 1 for y-axis
            if self.position[i] < 0:
                self.position[i] = 0
                if self.current_direction is not None:
                    self.current_direction[i] *= -1  # Reverse direction on the axis
            elif self.position[i] > FIELD_SIZE:
                self.position[i] = FIELD_SIZE
                if self.current_direction is not None:
                    self.current_direction[i] *= -1  # Reverse direction on the axis

    def reproduce(self):
        """Produce descendants."""
        descendants = []
        for _ in range(2):
            descendant = self.__class__(
                position=self.position.copy(),
                size=max(1.5, self.size + np.random.uniform(-SMALL_VARIATION, SMALL_VARIATION)),
                speed=max(0.5, self.speed + np.random.uniform(-SMALL_VARIATION, SMALL_VARIATION)),
                energy=self.energy / 2,  # Split energy between parent and descendants
                max_energy=self.max_energy,
                radius_of_view=max(3, self.radius_of_view + np.random.uniform(-LARGE_VARIATION, LARGE_VARIATION)),
                direction_change_interval=max(3, self.direction_change_interval + np.random.uniform(-LARGE_VARIATION, LARGE_VARIATION)),
                color=self.color,
                generation=self.generation + 1
            )
            descendant.eaten_count = 0
            self.energy /= 2  # Deduct energy for reproduction
            descendants.append(descendant)
        return descendants

class Herbivore(Animal):
    """Herbivore animal that eats food."""
    def update(self, environment):
        if self.is_alive():
            self.move(environment)
            self.handle_boundary()
            self.energy -= 0.5  # Energy decreases over time
            # Handle food consumption
            self.consume_food(environment.food_items)
            # Handle reproduction
            if self.energy >= self.max_energy:
                descendants = self.reproduce()
                environment.add_animals(descendants)

    def move(self, environment):
        """Move towards the closest food within radius of view, else move randomly."""
        closest_food = self.get_closest_food(environment.food_items)
        if closest_food:
            self.move_towards(closest_food.position)
        else:
            self.random_move()

    def get_closest_food(self, food_items):
        """Find the closest food within the animal's radius of view."""
        if not food_items:
            return None

        food_positions = np.array([food.position for food in food_items])
        distances = np.linalg.norm(food_positions - self.position, axis=1)
        within_view = distances <= self.radius_of_view

        if np.any(within_view):
            closest_food_idx = np.argmin(distances[within_view])
            return food_items[np.where(within_view)[0][closest_food_idx]]

        return None

    def move_towards(self, target_position):
        """Move the animal towards a specific position."""
        direction = target_position - self.position
        norm = np.linalg.norm(direction)
        if norm == 0:
            return  # Already at the target
        direction /= norm  # Normalize direction vector
        self.current_direction = direction  # Store the direction for potential reflection
        self.position += self.current_direction * self.speed

    def consume_food(self, food_items):
        """Increase energy when the animal consumes food."""
        closest_food = self.get_closest_food(food_items)
        if closest_food and distance(self.position, closest_food.position) < self.size:
            self.energy += FOOD_ENERGY_VALUE
            food_items.remove(closest_food)

class Carnivore(Animal):
    """Carnivore animal that eats other animals."""
    def update(self, environment):
        if self.is_alive():
            self.move(environment)
            self.handle_boundary()
            self.energy -= 0.5  # Energy decreases over time
            # Handle animal consumption
            self.consume_prey(environment.animals)
            # Handle reproduction
            if self.energy >= self.max_energy:
                descendants = self.reproduce()
                environment.add_animals(descendants)

    def move(self, environment):
        """Move towards the closest prey within radius of view, else move randomly."""
        closest_prey = self.get_closest_prey(environment.animals)
        if closest_prey:
            self.move_towards(closest_prey.position)
        else:
            self.random_move()

    def get_closest_prey(self, animals):
        """Find the closest prey within the animal's radius of view."""
        potential_prey = [animal for animal in animals if isinstance(animal, Herbivore) and animal != self]
        if not potential_prey:
            return None

        prey_positions = np.array([prey.position for prey in potential_prey])
        distances = np.linalg.norm(prey_positions - self.position, axis=1)
        within_view = distances <= self.radius_of_view

        if np.any(within_view):
            closest_prey_idx = np.argmin(distances[within_view])
            return potential_prey[np.where(within_view)[0][closest_prey_idx]]

        return None

    def move_towards(self, target_position):
        """Move the animal towards a specific position."""
        direction = target_position - self.position
        norm = np.linalg.norm(direction)
        if norm == 0:
            return  # Already at the target
        direction /= norm  # Normalize direction vector
        self.current_direction = direction  # Store the direction for potential reflection
        self.position += self.current_direction * self.speed

    def consume_prey(self, animals):
        """Consume prey if within reach."""
        for animal in animals:
            if animal is not self and isinstance(animal, Herbivore):
                dist = distance(self.position, animal.position)
                if dist < (self.size + animal.size):
                    if self.size > animal.size + SMALL_VARIATION * 2:
                        # Consume the prey
                        self.energy += animal.energy
                        self.eaten_count += 1
                        animals.remove(animal)
                        break

class Environment:
    """Class representing the simulation environment."""
    def __init__(self):
        self.field = Field(FIELD_SIZE)
        self.animals = []
        self.food_items = []
        self.steps_since_last_respawn = 0

    def add_animals(self, new_animals):
        """Add new animals to the environment."""
        self.animals.extend(new_animals)

    def add_food(self, food_items):
        """Add new food items to the environment."""
        self.food_items.extend(food_items)

    def update(self):
        """Update the state of the environment."""
        # Update animals
        for animal in self.animals[:]:
            animal.update(self)
            if not animal.is_alive():
                self.animals.remove(animal)
        # Respawn food if necessary
        self.steps_since_last_respawn += 1
        if self.steps_since_last_respawn >= FOOD_RESPAWN_INTERVAL:
            self.respawn_food()
            self.steps_since_last_respawn = 0

    def respawn_food(self):
        """Respawn food to ensure the total number of food items is always equal to MAX_FOOD_ITEMS."""
        num_missing_food = MAX_FOOD_ITEMS - len(self.food_items)
        for _ in range(num_missing_food):
            new_food_pos = np.random.uniform(0, self.field.size, size=2)
            self.food_items.append(Food(position=new_food_pos))

    def visualize(self, step):
        """Visualize the animals and food on the field."""
        fig, ax = plt.subplots(figsize=(6, 6))

        # Draw the animals
        for animal in self.animals:
            # Animal body
            circle = Circle(animal.position, animal.size, color=animal.color, alpha=0.6)
            ax.add_patch(circle)

            # Radius of view
            view_circle = Circle(animal.position, animal.radius_of_view, color=animal.color, fill=False, linestyle='--', alpha=0.3)
            ax.add_patch(view_circle)

            # Movement direction (if moving randomly or towards food)
            if animal.current_direction is not None:
                arrow_scale = animal.size * 3
                ax.arrow(
                    animal.position[0],
                    animal.position[1],
                    animal.current_direction[0] * arrow_scale,
                    animal.current_direction[1] * arrow_scale,
                    head_width=1, head_length=2, fc=animal.color, ec=animal.color
                )

            # Display energy, generation, and eaten count
            ax.text(
                animal.position[0],
                animal.position[1] + animal.size + 4,  # Adjusted position for better visibility
                f"E: {int(animal.energy)}\n"
                f"G: {animal.generation}\n"
                f"Eaten: {animal.eaten_count}\n"
                f"Size: {animal.size:.2f}",
                color='black',
                fontsize=7,
                ha='center'
            )

        # Draw the food items
        for food in self.food_items:
            triangle = RegularPolygon(food.position, numVertices=3, radius=1.5, orientation=np.pi / 2, color='green')
            ax.add_patch(triangle)

        # Set field limits and aspect ratio
        ax.set_xlim(0, self.field.size)
        ax.set_ylim(0, self.field.size)
        ax.set_aspect('equal', 'box')
        ax.set_title(f"Step: {step} | Animals: {len(self.animals)}")
        plt.tight_layout()
        plt.show()
        plt.close(fig)  # Close the figure to free memory

def simulate(environment, steps=1000):
    """Simulate the behavior of animals and food over a number of steps."""
    for step in range(1, steps + 1):
        if not environment.animals:
            print(f"All animals have perished by step {step}.")
            break

        environment.update()
        environment.visualize(step)
        time.sleep(0.05)  # Reduced pause for faster simulation
        clear_output(wait=True)

def initialize_simulation():
    """Initialize animals and food for the simulation."""
    environment = Environment()

    # Initialize animals
    herbivore1 = Herbivore(
        position=[FIELD_SIZE / 4, FIELD_SIZE / 4],
        size=2,
        speed=2,
        energy=100,
        max_energy=300,
        radius_of_view=20,
        direction_change_interval=10,
        color='blue'
    )
    carnivore1 = Carnivore(
        position=[3 * FIELD_SIZE / 4, 3 * FIELD_SIZE / 4],
        size=2.5,
        speed=2.5,
        energy=100,
        max_energy=300,
        radius_of_view=25,
        direction_change_interval=10,
        color='red'
    )
    environment.add_animals([herbivore1, carnivore1])

    # Generate initial random food positions
    food_items = [Food(position=np.random.uniform(0, FIELD_SIZE, size=2)) for _ in range(MAX_FOOD_ITEMS)]
    environment.add_food(food_items)

    return environment

# Run the simulation
environment = initialize_simulation()
simulate(environment, steps=1000)


KeyboardInterrupt: 